# Comparación entre esquemas de normalización para imágenes CT

- El pipeline [nnUNet](https://github.com/MIC-DKFZ/nnUNet) utiliza un esquema de normalización que no refleja el ventaneo o *windowing* realizado por los radiólogos al evaluar un estudio CT.
    - Para cada estudio, nnUNET realiza un clipping en los percentiles 0.5 y 99.5, y luego resta el promedio y divide por la desviación estándar.
    - Tanto los percentiles como el promedio y la desviación estándar se computan sobre el conjunto de voxeles *foreground* (en este caso, todos los voxeles anotados como lesión) del conjunto de entrenamiento. 
- El ventaneo o *windowing* consiste en un ajuste no lineal de contraste en un estudio CT, con el objetivo de identificar ciertos órganos o tejidos de interés.
    - Cada ventana posee un centro o nivel (*window_level* o WL) y un ancho (*window_width* o WW) en Unidades de Hounsfield (HU).
    - Los voxeles con valores fuera de la ventana definida por su centro y ancho, se clipean a los extremos de dicha ventana.
    - Luego, para desplegar la imagen en pantalla, se mapean linealmente los valores de los voxeles dentro de la ventana al rango de colores/valores en escala de grises disponibles en el visualizador (típicamente al rango 0-255, donde 0 es negro y 255 es blanco).
    - Algunos ejemplos de ventanas son (valores por defecto utilizados en 3D slicer):
        - CT-Brain: WL=50, WW=100
        - CT-Abdomen: WL=40, WW=350
        - CT-Lung: WL=-500, WW=1400
        - CT-Air: WL=-426, WW=1000
        - CT-Bone: WL=400, WW=1000
- Se propone reemplazar el esquema de normalización nnUNet por uno que emule el ventaneo. En particular:
    - Se realiza clipping de acuerdo a los extremos definidos por la ventana.
    - Se normalizan los valores dentro de la ventana al rango 0-1.
- La comparación entre los esquemas de normalización muestran que:
    - La implementación propuesta de *windowing* coincide visualmente con la integrada en el software 3D slicer, tanto para abdomen como para pulmón.
    - La normalización basada en *windowing* es superior a la integrada por el pipeline nnUnet, pues permite visualizar de mejor manera los órganos y tejidos de interés en abdomen y pulmón.
    - ¿La normalización propuesta permite identificar visualmente las lesiones anotadas como ground truth? Pareciera que, aun con la normalización basada en ventaneo, el delineamiento de las lesiones anotadas está fuertemente influenciado por las estructuras visibles en la imagen PET, y no en la imagen CT. 

![Comparación en abdomen](resources/2023_10_23_Comparison_of_normalization_schemas-abdomen.png)
**Comparación de esquemas de normalización para zona de abdomen.** Cada columna corresponde a un corte distinto en el plano axial para un paciente con melanoma. De arriba hacia abajo, cada fila corresponde a: **i) nnUnet normalization:** normalización aplicada por nnUnet, **ii) custom abdomen windowing:** normalización propuesta para ventana de abdomen, **iii) 3dSlicer abdomen windowing:** visualización en 3D slicer utilizando ventana de abdomen, **iv) 3dSlicer abdomen windowing + SEG:** visualización en 3D slicer utilizando ventana de abdomen, con máscara de segmentación ground truth superpuesta (verde), **v) PET:** visualización en 3D slicer de estudio PET correspondiente.

![Comparación en pulmón](resources/2023_10_23_Comparison_of_normalization_schemas-lung.png)
**Comparación de esquemas de normalización para zona de pulmón.** Cada columna corresponde a un corte distinto en el plano axial para un paciente con cáncer de pulmón. De arriba hacia abajo, cada fila corresponde a: **i) nnUnet normalization:** normalización aplicada por nnUnet, **ii) custom lung windowing:** normalización propuesta para ventana de pulmón, **iii) 3dSlicer lung windowing:** visualización en 3D slicer utilizando ventana de pulmón, **iv) 3dSlicer lung windowing + SEG:** visualización en 3D slicer utilizando ventana de pulmón, con máscara de segmentación ground truth superpuesta (verde), **v) PET:** visualización en 3D slicer de estudio PET correspondiente.

![Visualización lesiones anotadas en abdomen](resources/2023_10_23_Comparison_of_normalization_schemas_abdomen_detailed.png)**Visualización de lesiones ground truth en ventana de abdomen.** Cada columna corresponde a un corte distinto en el plano axial para un paciente con melanoma. De arriba hacia abajo, cada fila corresponde a: **i) custom abdomen windowing:** normalización propuesta para ventana de abdomen, **ii) 3dSlicer abdomen windowing + SEG:** visualización en 3D slicer utilizando ventana de abdomen, con máscara de segmentación ground truth superpuesta (verde).

![Visualización lesiones anotadas en pulmón](resources/2023_10_23_Comparison_of_normalization_schemas_lung_detailed.png)**Visualización de lesiones ground truth en ventana de pulmón.** Cada columna corresponde a un corte distinto en el plano axial para un paciente con cáncer de pulmón. De arriba hacia abajo, cada fila corresponde a: **i) custom lung windowing:** normalización propuesta para ventana de pulmón, **ii) 3dSlicer lung windowing + SEG:** visualización en 3D slicer utilizando ventana de pulmón, con máscara de segmentación ground truth superpuesta (verde).